In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**LSTM**

In [ ]:
import pandas as pd
data = pd.read_csv("/kaggle/input/datasetartikel/Data Fix.xlsx - Sheet1.csv")

In [ ]:
data.dropna(axis=0, subset=['Article Content','Kategori'], inplace=True)
data.head()

In [ ]:
data[130:140]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X_train,X_test, y_train, y_test = train_test_split(data['Article Content'],data['Kategori'],train_size=0.8,random_state=42)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='#$$%&()*+,-./:;<=>?@[\]^_{|}~', lower=True)
tokenizer.fit_on_texts(data['Article Content'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(data['Article Content'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

In [ ]:
y = pd.get_dummies(data['Kategori']).values
print('Shape of label tensor:', y.shape)

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.10, random_state = 42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 30
batch_size = 64

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

In [ ]:
y_pred = model.predict(X_test)
y_pred.shape

**RNN**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dataset_train = pd.read_csv("/kaggle/input/datasetartikel/Data Fix.xlsx - Sheet1.csv")
training_set = dataset_train.iloc[:, 1:2].values
dataset_train.head()

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, SimpleRNN, Dense

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(SimpleRNN(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 30
batch_size = 64

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

**CNN**

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Activation,  Dropout 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv("/kaggle/input/datasetartikel/Data Fix.xlsx - Sheet1.csv")
data.head()

In [ ]:
text = data["Article Content"].tolist()
print(len(text))

In [ ]:
#Case Folding
import pandas
data['Article Content'] = data['Article Content'].str.lower()

print('Case Folding Result : \n')
print(data['Article Content'].head(5))
print('\n\n\n')

In [ ]:
token = Tokenizer()
token.fit_on_texts(text)

In [ ]:
token.index_word

In [ ]:
vocal = len(token.index_word)+1
vocal

In [ ]:
x =['aku lagi galau nih']
token.texts_to_sequences(x)

In [ ]:
encode_text = token.texts_to_sequences(text)
# print(encode_text)

In [ ]:
max_kata = 100
X = pad_sequences(encode_text, maxlen = max_kata, padding="post")
X

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.10, random_state = 42)


In [ ]:
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [ ]:
#Model CNN
vec_size=300
model = Sequential()
model.add(Embedding(vocal, vec_size, input_length=max_kata))
model.add(Conv1D(64,8, activation="relu"))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))

model.add(Dense(8, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(16, activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation="softmax"))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)